In [55]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="")
index = pc.Index("breakout-ai-index")
import os
from langchain_openai import OpenAIEmbeddings

os.environ['OPENAI_API_KEY'] = ''
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)

In [77]:
import mercury as mr

# add text widget
my_selection_stock = mr.Select(value = "NIFTY 50", choices = ["NIFTY 50", "Reliance Industries Limited", "Adani Enterprises Limited", "Custom"], label = "Select stock")

mercury.Select

In [111]:
if my_selection_stock.value == "Custom":
    stock_name = mr.Text(label="Stock code on Yahoo Finance", value="^NSEI")

mercury.Text

In [59]:
import mercury as mr
 
# add text widget
my_selection_st = mr.Select(value = "Daily strategy", choices = ["Daily strategy", "Minute strategy"], label = "Select type")

mercury.Select

In [60]:
#print(f"Strategy will be executed as a {my_selection_st.value}.")

In [61]:
if my_selection_st.value == "Daily strategy":
    your_range = mr.Range(value=[2010, 2024], min=2010, max=2024, label="Your backtesting range", step=1)
    
    #access widget value in the code
    #print(f"Your strategy backtesting starts on 01-01-{your_range.value[0]} ends on 01-01-{your_range.value[1]}.")

mercury.Range

In [62]:
no_of_stocks = mr.Slider(value=1, min=0, max=10, label="Number of stocks per slot: ", step=1)

mercury.Slider

In [112]:
import yfinance as yf
import pandas as pd


if my_selection_stock.value == "NIFTY 50":
    symbol = "^NSEI"
elif my_selection_stock.value == "Adani Enterprises Limited":
    symbol = "ADANIENT.NS"
    
elif my_selection_stock.value == "Reliance Industries Limited":
    symbol = "RELIANCE.NS"
    
elif my_selection_stock.value == "Custom":
    symbol = stock_name.value
    
if my_selection_st.value == "Daily strategy":
    interval = "1d"
    start_date = f"{your_range.value[0]}-01-01"
    end_date = f"{your_range.value[1]}-01-01"
    df = yf.download(tickers=symbol, interval=interval, start=start_date, end=end_date)


elif my_selection_st.value == "Minute strategy":
    interval="1m"
    df = yf.download(tickers=symbol, interval=interval)

[*********************100%%**********************]  1 of 1 completed


In [113]:
from IPython.display import display, HTML

display(HTML(df.head(5).to_html()))
print("to")
display(HTML(df.tail(5).to_html()))

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,13.275,13.348,13.163,13.346,12.164504,2117000
2010-01-05,13.277,13.403,13.250,13.403,12.216455,1906000
2010-01-06,13.250,13.510,13.250,13.463,12.271146,1293000
2010-01-07,13.357,13.689,13.350,13.678,12.467112,2944000
2010-01-08,13.402,13.750,13.402,13.719,12.504481,1712000


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-22,66.360001,67.160004,66.120003,67.000000,67.000000,1601800
2023-12-26,67.120003,67.180000,66.519997,66.720001,66.720001,1412900
2023-12-27,66.970001,67.440002,66.800003,67.040001,67.040001,1789900
2023-12-28,66.830002,67.360001,66.709999,67.220001,67.220001,1880600
2023-12-29,67.220001,67.250000,66.730003,67.110001,67.110001,796500


In [65]:
strategy = mr.Text(value="my strategy is the basic sma 200 vs sma 50.", label="describe your strategy here:", rows=4)

mercury.Text

In [66]:
get_ans = mr.Button(label="Get Backtested result", style="primary")
 
# after button clicked this will be true
# but it will be true only once!

mercury.Button

In [67]:
strategy_name="SMA_CrossOver"
strategy_code="""import backtrader as bt

class SMA_CrossOver(bt.Strategy):
    params = (('fast', 50), ('slow', 200),)

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    def __init__(self):
        self.sma1 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.p.fast)
        self.sma2 = bt.indicators.SimpleMovingAverage(self.data.close, period=self.p.slow)
        self.crossover = bt.indicators.CrossOver(self.sma1, self.sma2)

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.log('BUY CREATE, %.2f' % self.data.close[0])
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.log('SELL CREATE, %.2f' % self.data.close[0])
            self.close()  # close long position
"""

In [115]:
if get_ans.clicked:
    retry = True
    while retry:
        try:
            
            from langchain.tools.retriever import create_retriever_tool
            backtraderSearch = create_retriever_tool(
                retriever,
                "backtrader_search",
                "This tool contains the Documentation for the backtrader library which we are going to be using, when you are given a strategy, use this tool to see what indicators you are going to use, and how will you make a class using those indicators.",
            )
            from langchain.agents import AgentType, initialize_agent, load_tools
            from langchain_openai import ChatOpenAI, OpenAI
            tools = [backtraderSearch]
            from langchain_openai import ChatOpenAI

            model = ChatOpenAI(model="gpt-4o-mini")
            from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

            prompt = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        """hey I hope you are doing well, think like a quantitative coder now for a big stock market company. You will always be given a trading strategy as an input, the data will be provided to you, so you only hae to take help of the backtrader_search tool to see how will you make the class which I will later copy and paste and run it using backtrader api. Keep sure you are only making the class and nothing else. Also make the class such that when I run the cerebro engine I see what is hapening every day, that is after you are done with capturing the logic, make logs of the trade every day, example: class TestStrategy(bt.Strategy):

                def log(self, txt, dt=None):
                    ''' Logging function fot this strategy'''
                    dt = dt or self.datas[0].datetime.date(0)
                    print('%s, %s' % (dt.isoformat(), txt))

                def __init__(self):
                    # Keep a reference to the "close" line in the data[0] dataseries
                    self.dataclose = self.datas[0].close

                def next(self):
                    # Simply log the closing price of the series from the reference
                    self.log('Close, %.2f' % self.dataclose[0])

                    if self.dataclose[0] < self.dataclose[-1]:
                        # current close less than previous close

                        if self.dataclose[-1] < self.dataclose[-2]:
                            # previous close less than the previous close

                            # BUY, BUY, BUY!!! (with all possible default parameters)
                            self.log('BUY CREATE, %.2f' % self.dataclose[0])
                            self.buy()
                            
                            see here in this very basic stategy we can see that how the buy/sell is printed, you have to do this for every class you make. Also make sure your position is always there, that is if you are at a buy position you can only sell, if you are at a sell position you can only buy next. Also make sure that array don't go out of range and also the function data.close is used properly. Please generate good code as it is very important to my company.""",
                    ),
                    ("user", "{input}"),
                    MessagesPlaceholder(variable_name="agent_scratchpad"),
                ]
            )
            llm_with_tools = model.bind_tools(tools)
            from langchain.agents.format_scratchpad.openai_tools import (
                format_to_openai_tool_messages,
            )
            from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

            agent = (
                {
                    "input": lambda x: x["input"],
                    "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                        x["intermediate_steps"]
                    ),
                }
                | prompt
                | llm_with_tools
                | OpenAIToolsAgentOutputParser()
            )
            from langchain.agents import AgentExecutor

            agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
            
            ans = list(agent_executor.stream({"input": strategy.value}))[-1]['output']
            start = ans.find("python")
            if start != -1:
                # Find the first occurrence of triple quotes after the starting point
                end = ans.find("```", start)
                if end != -1:
                    # Extract the content between the starting point and the triple quotes
                    content = ans[start+len("python"):end]
                    ans = content
                strategy_code = ans
                
            strategy_name = ans
            start = strategy_name.find("class ")
            if start != -1:
                # Find the first occurrence of triple quotes after the starting point
                end = strategy_name.find("(bt.Strategy)", start)
                if end != -1:
                    # Extract the content between the starting point and the triple quotes
                    content = strategy_name[start+len("class "):end]
                    strategy_name = content
            retry = False
        
        except Exception as e:
            print(f"Error occurred: {e}. Retrying...")
        
    
    #print(strategy_code)
    #print(strategy_name)

In [69]:
import backtrader as bt

In [70]:
exec(strategy_code)

In [71]:
data_feed = data_feed = bt.feeds.PandasData(dataname=df)

In [72]:
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__


In [73]:
exec((f"""cerebro = bt.Cerebro()
cerebro.addstrategy({strategy_name})
cerebro.adddata(data_feed)
cerebro.broker.setcash(100000.0)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())"""))

In [74]:
profit_per_stock = cerebro.broker.getvalue() - 100000
profit_give_stock = profit_per_stock*(no_of_stocks.value)
profit_percent = profit_per_stock/100

In [75]:
import math
mr.NumberBox([
    mr.NumberBox(data=f'{math.floor(profit_per_stock)}₹', title="Profit per stock"),
    mr.NumberBox(data=f'{math.floor(profit_give_stock)}₹', title="Total Profit"),
    mr.NumberBox(data=f'{(math.floor(profit_percent))/10}%', title="Profit Percentage")
])

In [76]:
from IPython.display import display
import matplotlib.pyplot as plt

# Assuming you have your strategy and data loaded into cerebro
# Replace `cerebro.plot()` with the following:


fig = cerebro.plot(style='bar', volume=False)[0][0]
fig.set_size_inches(12, 8)


# Use IPython's display to render the plot in the notebook
display(fig)

# Close the plot to avoid duplication in some environments
plt.close(fig)


<IPython.core.display.Javascript object>

<Figure size 1200x800 with 4 Axes>